In [142]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [143]:
N_EPOCHS = 70
N_TRIALS = 3
BATCH_SIZE = 256

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [152]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

# x_train = x_train / 255
# x_val = x_val / 255
# x_test = x_test / 255

In [153]:
noise = 30
x_train = np.clip(x_train + np.random.normal(0, noise, x_train.shape), 0, 255) / 255
x_val = np.clip(x_val + np.random.normal(0, noise, x_val.shape), 0, 255) / 255
x_test = np.clip(x_test + np.random.normal(0, noise, x_test.shape), 0, 255) / 255

In [154]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [155]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)
            samples_ids = np.random.choice(range(train_size), n_samples, replace=False)

            model.fit(
                x_train[samples_ids], y_train[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE, verbose=1
            )
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)
        
        model_scores.append(accuracy)

    results[test_model] = model_scores

test_model_1


  0%|          | 0/3 [00:00<?, ?it/s]

6/6 [==============================] - 0s 23ms/step - loss: 1.7176 - accuracy: 0.4553


 33%|███▎      | 1/3 [03:09<06:19, 189.81s/it]

6/6 [==============================] - 0s 22ms/step - loss: 1.6503 - accuracy: 0.4600


 67%|██████▋   | 2/3 [06:18<03:08, 188.88s/it]

6/6 [==============================] - 0s 21ms/step - loss: 1.6443 - accuracy: 0.4653


100%|██████████| 3/3 [09:43<00:00, 194.53s/it]


In [160]:
results

{'test_model_1': [0.45533332228660583,
  0.46000000834465027,
  0.46533334255218506]}

In [159]:
sum(results["test_model_1"]) / 3

0.4602222243944804